In [ ]:
#######################################################################
##  Made by: Dr. Keungoui Kim
##  Title: Funding data import from Dimensions
##  goal : 
##  Data set:  
##  Time Span:
##  Variables
##      Input: 
##      Output: 
##  Time-stamp: #  "Sun Jan 26 01:47:34 2020":  edited by awe kim ; code
##  Notice :
#######################################################################

In [8]:
### Library
import pandas as pd
import numpy as np
import math

### Directory Setting for Export
#dir = "E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/" # Home
dir = "D:/Google Drive(awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/" # HGU

In [9]:
### API log in
import dimcli
dimcli.login(key="792DDFAFCCA7478D8F37159F274A2783",
             endpoint="https://app.dimensions.ai/api/dsl/v2")

dsl = dimcli.Dsl()

Dimcli - Dimensions API Client (v0.9.9.1)
Connected to: <https://app.dimensions.ai/api/dsl/v2> - DSL v2.5
Method: manual login


In [ ]:
# Extract Grant data
# Extract Funding data 
# What funding initiates science -> technology

In [13]:
years = ['2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'] #, '2000','2001','2002','2003','2004','2005','2006','2007','2008','2009', '2010'
offices = ['EP'] # 'WO', 'US', 'EP'

#### Create grant data set

In [54]:
### Load pub_grant data set to merge it to pub_grant_all
import re

for office in offices:
    # Filter list of files "pub_grant_df"
    file_list = os.listdir(dir)
    r = re.compile(".*pub_grant_list"+".*"+office+".csv")
    filtered_list = list(filter(r.match, file_list))
    filtered_list

    pub_grant_all = pd.DataFrame()
    for i in range(len(filtered_list)):
        temp_df = pd.read_csv(dir+filtered_list[i])
        temp_df.dropna(inplace=True)
        pub_grant_all = pd.concat([pub_grant_all, temp_df], axis=0)

    pub_grant_all.to_csv(dir+"publication/pub_grant_list_"+office+".csv", index=False)


In [126]:
### 
for office in offices:
    pub_grant_all = pd.read_csv(dir+"publication/pub_grant_list_"+office+".csv")
    pub_grant_list = pd.DataFrame(pub_grant_all['supporting_grant_ids'].unique(), columns=['supporting_grant_ids'])

    inloop = math.ceil(pub_grant_list.shape[0]/500)+1
    pub_grant_df = pd.DataFrame() # Create empty data.frame  
    for j in range(1, inloop):
        intemp = pub_grant_list.iloc[0:499,]
        query = """search grants
                    where id in """+'''['''+','.join(f'"{x}"' for x in intemp['supporting_grant_ids'].to_list())+''']'''+"""
                    return grants"""
        intemp_data = dsl.query_iterative(query)
        intemp_data = intemp_data.as_dataframe()    

        pub_grant_df = pd.concat([pub_grant_df, intemp_data], axis=0)

        pub_grant_list = pub_grant_list.drop(pub_grant_list.index[0:499])   

    pub_grant_df.to_csv(dir+"publication/pub_grant_df_"+office+".csv", index=False)

    pub_funders_df = pd.DataFrame()
    for i in range(pub_grant_df.shape[0]):
        temp_data = pd.DataFrame.from_records(pub_grant_df['funder_orgs'][i])
        temp_data['grant_id'] = pub_grant_df['id'][i]
        pub_funders_df = pd.concat([pub_funders_df, temp_data], axis=0)
        
    pub_funders_df.to_csv(dir+"publication/pub_funders_df_"+office+".csv", index=False)
          

Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (2.53s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (2.56s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (2.76s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (2.63s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (2.70s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (10.92s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (3.08s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (3.03s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (8.85s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (3.61s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (2.99s)
===

In [ ]:
pub_funders_df.head()

,acronym,city_name,country_name,id,latitude,linkout,longitude,name,state_name,types
0,NHLBI,Bethesda,United States,grant.2725865,39.004280,[http://www.nhlbi.nih.gov/],-77.100945,National Heart Lung and Blood Institute,Maryland,[Government]
0,NIAID,Bethesda,United States,grant.2723057,39.066647,[http://www.niaid.nih.gov/Pages/default.aspx],-77.111830,National Institute of Allergy and Infectious D...,Maryland,[Government]
0,NIAID,Bethesda,United States,grant.2723043,39.066647,[http://www.niaid.nih.gov/Pages/default.aspx],-77.111830,National Institute of Allergy and Infectious D...,Maryland,[Government]
0,NIAID,Bethesda,United States,grant.2723022,39.066647,[http://www.niaid.nih.gov/Pages/default.aspx],-77.111830,National Institute of Allergy and Infectious D...,Maryland,[Government]
0,EPA,Washington D.C.,United States,grant.4597979,38.893740,[http://www2.epa.gov/],-77.030430,Environmental Protection Agency,District of Columbia,[Government]
...,...,...,...,...,...,...,...,...,...,...
0,NHLBI,Bethesda,United States,grant.2635546,39.004280,[http://www.nhlbi.nih.gov/],-77.100945,National Heart Lung and Blood Institute,Maryland,[Government]
0,NHLBI,Bethesda,United States,grant.2529595,39.004280,[http://www.nhlbi.nih.gov/],-77.100945,National Heart Lung and Blood Institute,Maryland,[Government]
0,NIAID,Bethesda,United States,grant.2449219,39.066647,[http://www.niaid.nih.gov/Pages/default.aspx],-77.111830,National Institute of Allergy and Infectious D...,Maryland,[Government]
0,NHLBI,Bethesda,United States,grant.12966897,39.004280,[http://www.nhlbi.nih.gov/],-77.100945,National Heart Lung and Blood Institute,Maryland,[Government]


In [127]:
pub_grant_df.shape

(147145, 11)

In [128]:
pub_grant_df['id'].nunique()

147145

In [69]:
grant_data_df.head()

,id,title,active_year,end_date,funder_org_name,funder_orgs,grant_number,language,original_title,start_date,start_year
0,grant.9852804,"Multi-scale, multi-modal X-ray imaging using s...","[2023, 2024, 2025, 2026]",2026-12-30,Australian Research Council,"[{'acronym': 'ARC', 'city_name': 'Canberra', '...",DE220101402,en,"Multi-scale, multi-modal X-ray imaging using s...",2023-12-31,2023
1,grant.12981162,Defining the Pathogenic Role of Neural Crest C...,"[2023, 2024, 2025]",2025-11-30,TSC Alliance,"[{'acronym': 'TS Alliance', 'city_name': 'Silv...",990240,en,Defining the Pathogenic Role of Neural Crest C...,2023-12-01,2023
2,grant.12956342,Network slicing and fairness over optical tran...,"[2023, 2024]",2024-11-30,São Paulo Research Foundation,"[{'acronym': 'FAPESP', 'city_name': 'São Paulo...",19/26130-4,en,Network slicing and fairness over optical tran...,2023-12-01,2023
3,grant.12930187,3D printing multifunctional devices without in...,"[2023, 2024, 2025]",2025-05-30,Engineering and Physical Sciences Research Cou...,"[{'acronym': 'EPSRC', 'city_name': 'Swindon', ...",EP/W034409/1,en,3D printing multifunctional devices without in...,2023-12-01,2023
4,grant.12915816,Iron gall ink challenges. Methodologies for id...,"[2023, 2024]",2024-01-31,Fundação para a Ciência e Tecnologia,"[{'acronym': 'FCT', 'city_name': 'Lisbon', 'co...",COVID/BD/153467/2023,en,Iron gall ink challenges. Methodologies for id...,2023-12-01,2023


In [71]:
grant_data_df['funder_orgs'][0]

[{'acronym': 'ARC',
  'city_name': 'Canberra',
  'country_name': 'Australia',
  'id': 'grid.413452.5',
  'latitude': -35.282425,
  'linkout': ['http://www.arc.gov.au/'],
  'longitude': 149.12405,
  'name': 'Australian Research Council',
  'state_name': 'Australian Capital Territory',
  'types': ['Government']}]